In [1]:
import sys
import numpy as np
import os
current_directory = os.getcwd()
relative_path1 = '../build/env_simulator'
relative_path2 = '../build/cilqr'
file_path1 = os.path.join(current_directory, relative_path1)
file_path2 = os.path.join(current_directory, relative_path2)
sys.path.append(file_path1)
sys.path.append(file_path2)
import pybind_env_simulator

relative_path3 = '../env_simulator'
file_path3 = os.path.join(current_directory, relative_path3)
sys.path.append(file_path3)
from py_lib import py_env_sim
from py_lib import cilqr_plot
import pybind_ilqr
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import WheelZoomTool
import json
from IPython.display import display, HTML
display(HTML("<style> div.output_scroll { height: auto !important; max-height: none !important; } </style>"))

In [2]:
from sympy import symbols, diff, sin, cos, exp
w = symbols('w')
x0 = symbols('x0')
theta = symbols('theta')
alpha = symbols('alpha')
L = symbols('L')

L = w * (x0 + sin(theta -alpha) * L)*(x0 + sin(theta -alpha) * L)
L_x = diff(L, theta)
L_x_x = diff(L_x, theta)
print("L_x:",L_x)
print("L_x_x:",L_x_x)

L_x: 2*L*w*(-L*sin(alpha - theta) + x0)*cos(alpha - theta)
L_x_x: 2*L**2*w*cos(alpha - theta)**2 + 2*L*w*(-L*sin(alpha - theta) + x0)*sin(alpha - theta)


## 横向直线不居中 1 P0

In [3]:
def show_motion_tree_case(motion): 
    traj_trees = motion.get_trajectory_tree()

    print("trajectory tree branch size is: ", len(traj_trees))
    env = motion.get_env()
    planning_origin = motion.get_planning_origin()

    pts = env.get_all_center_points()
    obs = env.get_all_obstacle()
    planning_init = {}

    planning_init['x'] = planning_origin.position.x
    planning_init['y'] = planning_origin.position.y
    planning_init['theta'] = planning_origin.theta

    trajs = motion.get_init_trajectory()
    traj_list = []
    for traj in trajs:
        pt = {}
        pt['x'] = traj.position.x
        pt['y'] = traj.position.y
        pt['theta'] = traj.theta
        traj_list.append(pt)
    
    lon_debug = motion.get_lon_debug_tree()
    lat_debug = motion.get_lat_debug_tree()
    traj_list_list = []
    j = 0
    for traj_tree in traj_trees:  
        traj_new_list = []
        index = 0
        for traj in traj_tree:
            pt = {}
            pt['x'] = traj.position.x
            pt['y'] = traj.position.y
            pt['theta'] = traj.theta
            pt['v'] = traj.velocity
            pt['omega'] = traj.omega
            pt['ref_omega'] = lat_debug[j][index].ref_omega
            pt['acc'] = traj.acceleration
            pt['lat_dis'] = lat_debug[j][index].lat_dis
            pt['acc_lat'] = traj.velocity * traj.omega
            pt['acc_lat_ref'] = lat_debug[j][index].ref_lat_acc
            pt['ref_vel'] = lon_debug[j][index].v_ref
            pt['ref_acc'] = lon_debug[j][index].a_ref
            pt['match_point'] = lat_debug[j][index].match_point
            pt['jerk'] = traj.jerk
            pt['omega_dot'] = traj.omega_dot
            index = index + 1
            traj_new_list.append(pt)
        traj_list_list.append(traj_new_list)
        j = j+1
    cilqr_plot.plot_tree_cilqr_constraint(pts,obs,planning_init,traj_list, traj_list_list,[])

In [4]:
motion = pybind_ilqr.LateralLongitudinalMotion()
point_init = pybind_ilqr.PlanningPoint()
point_init.position.x = 10.0
point_init.position.y = 4.0
point_init.theta = 0.00
point_init.velocity = 15.0
point_init.acceleration  = 0.6

path = str(current_directory)+str("/../env_simulator/data/test_case_1_straight_lanes_cruise.json")
motion.init(path,point_init)

motion.execute_tree()

show_motion_tree_case(motion)

---- loading test_case_1_straight_case_1 case ----
trajectory tree branch size is:  1
---- loading relative -1 lane 
---- loading relative 0 lane 
---- loading relative 1 lane 
---- loading obstacle 100
Field 'road_edges' does not exist.
 r_4 is : 1.13033
j_opt: 298.743
------- best alpha: 1
------- best alpha: 1
----------enable lat 1
------- best alpha: 1
------- best alpha: 0.683013
------- best alpha: 0.909091
------- best alpha: 0.909091
 iter times: 5
 stop search 
 **************** ilqr tree time: 9.31868 ms 
 tree trajectory finish


Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

## 弯道 1 P0

In [5]:
point_init.position.x = 10.0
point_init.position.y = 5.25
point_init.position.y = 2.25
point_init.theta = 0.1
point_init.velocity = 22.22
point_init.acceleration  = 0.5

path = str(current_directory)+str("/../env_simulator/data/curva_lanes_case_1.json")
motion.init(path,point_init)

motion.execute_tree()

show_motion_tree_case(motion)

trajectory tree branch size is:  1
---- loading straight_case_1 case ----
---- loading relative -1 lane 
---- loading relative 0 lane 
---- loading relative 1 lane 
Field 'road_edges' does not exist.
 r_4 is : 1.13033
j_opt: 2219.58
------- best alpha: 1
------- best alpha: 1
----------enable lat 1
------- best alpha: 1
------- best alpha: 0.683013
------- best alpha: 0.909091
------- best alpha: 0.683013
------- best alpha: 0.424098
 iter times: 6
 stop search 
 **************** ilqr tree time: 11.1373 ms 
 tree trajectory finish


Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

## 弯道 2  P1

In [6]:
point_init.position.x = 10.0
point_init.position.y = 5.25
point_init.theta = 0.1
point_init.velocity = 10.22
point_init.acceleration  = 0.5

path = str(current_directory)+str("/../env_simulator/data/curva_lanes_case_3.json")
motion.init(path,point_init)

motion.execute_tree()

show_motion_tree_case(motion)

trajectory tree branch size is:  1
---- loading straight_case_1 case ----
---- loading relative 0 lane 
Field 'road_edges' does not exist.
 r_4 is : 1.13033
j_opt: 15.2386
------- best alpha: 1
------- best alpha: 1
----------enable lat 1
------- best alpha: 1
------- best alpha: 0.683013
------- best alpha: 0.683013
------- best alpha: 0.683013
------- best alpha: 0.909091
------- best alpha: 0.909091
 iter times: 7
 stop search 
 **************** ilqr tree time: 12.4016 ms 
 tree trajectory finish


Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

In [7]:
point_init.position.x = 10.0
point_init.position.y = 1.25
point_init.theta = 0.1
point_init.velocity = 10.22
point_init.acceleration  = 0.5

path = str(current_directory)+str("/../env_simulator/data/curva_lanes_case_3.json")
motion.init(path,point_init)

motion.execute_tree()

show_motion_tree_case(motion)

---- loading straight_case_1 case ----
trajectory tree branch size is:  1
---- loading relative 0 lane 
Field 'road_edges' does not exist.
 r_4 is : 1.13033
j_opt: 15.2386
------- best alpha: 1
------- best alpha: 1
----------enable lat 1
------- best alpha: 1
------- best alpha: 0.683013
------- best alpha: 0.909091
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 0.424098
 iter times: 8
 stop search 
 **************** ilqr tree time: 14.0013 ms 
 tree trajectory finish


Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

## 弯道 3  P0 近似快速变道

In [8]:
point_init.position.x = 30
point_init.position.y = 5.8
point_init.theta = -0.1
point_init.velocity = 14.22
point_init.acceleration  = 0.1

path = str(current_directory)+str("/../env_simulator/data/curva_lanes_case_3.json")
motion.init(path,point_init)

motion.execute_tree()

show_motion_tree_case(motion)

trajectory tree branch size is:  1
---- loading straight_case_1 case ----
---- loading relative 0 lane 
Field 'road_edges' does not exist.
 r_4 is : 1.13033
j_opt: 198.589
------- best alpha: 1
------- best alpha: 1
----------enable lat 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 0.092296
 iter times: 19
 stop search 
 **************** ilqr tree time: 35.61 ms 
 tree trajectory finish


Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

## 弯道 3.1  P0 近似无加速变道

In [15]:
point_init.position.x = 30
point_init.position.y = 5.8
point_init.theta = 0.0
point_init.velocity = 14.22
point_init.acceleration  = 0.0

path = str(current_directory)+str("/../env_simulator/data/curva_lanes_case_3.json")
motion.init(path,point_init)

motion.execute_tree()

show_motion_tree_case(motion)

trajectory tree branch size is: ---- loading straight_case_1 case ----
---- loading relative 0 lane 
Field 'road_edges' does not exist.
 r_4 is : 1.13033
j_opt: 549.8
------- best alpha: 1
------- best alpha: 0.909091
----------enable lat 1
------- best alpha: 0.000443805
------- best alpha: 0.217629
------- best alpha: 0.683013
------- best alpha: 1
------- best alpha: 0.909091
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
 iter times: 15
 stop search 
 **************** ilqr tree time: 31.4648 ms 
 tree trajectory finish
 1


Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

## 弯道 4  P0 弯道通行

In [9]:
point_init.position.x = 40.0
point_init.position.y = 3.0
point_init.theta = 0.2
point_init.velocity = 16.22
point_init.acceleration  = 0.5

path = str(current_directory)+str("/../env_simulator/data/curva_lanes_case_3.json")
motion.init(path,point_init)

motion.execute_tree()

show_motion_tree_case(motion)

trajectory tree branch size is: ---- loading straight_case_1 case ----
 1
---- loading relative 0 lane 
Field 'road_edges' does not exist.
 r_4 is : 1.13033
j_opt: 4958.97
------- best alpha: 1
------- best alpha: 1
----------enable lat 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 0.683013
------- best alpha: 1
------- best alpha: 0.909091
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 0.683013
------- best alpha: 1
------- best alpha: 0.424098
------- best alpha: 1
 iter times: 20
 stop search 
 **************** ilqr tree time: 35.9076 ms 
 tree trajectory finish


Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

## 弯道 4.1  P0 弯道通行

In [10]:
point_init.position.x = 40.0
point_init.position.y = 3.0
point_init.theta = 0.15
point_init.velocity = 16.22
point_init.acceleration  = 0.5

path = str(current_directory)+str("/../env_simulator/data/curva_lanes_case_3.json")
motion.init(path,point_init)

motion.execute_tree()

show_motion_tree_case(motion)

trajectory tree branch size is:  1
---- loading straight_case_1 case ----
---- loading relative 0 lane 
Field 'road_edges' does not exist.
 r_4 is : 1.13033
j_opt: 4958.97
------- best alpha: 1
------- best alpha: 1
----------enable lat 1
------- best alpha: 1
------- best alpha: 0.217629
------- best alpha: 1
------- best alpha: 0.683013
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 0.683013
------- best alpha: 0.0323492
 iter times: 21
 stop search 
 **************** ilqr tree time: 37.5129 ms 
 tree trajectory finish


Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

## 弯道 5  P1 特殊场景

In [11]:
point_init.position.x = 10.0
point_init.position.y = 6.0
point_init.theta = 0.2
point_init.velocity = 5.67
point_init.acceleration  = 0.5

path = str(current_directory)+str("/../env_simulator/data/curva_lanes_case_2.json")
motion.init(path,point_init)

motion.execute_tree()

show_motion_tree_case(motion)

trajectory tree branch size is:  1
---- loading straight_case_1 case ----
---- loading relative 0 lane 
Field 'road_edges' does not exist.
 r_4 is : 1.13033
j_opt: 63.8194
------- best alpha: 1
------- best alpha: 1
----------enable lat 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 0.683013
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 0.683013
------- best alpha: 0.092296
 iter times: 25
 stop search 
 **************** ilqr tree time: 44.4041 ms 
 tree trajectory finish


Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

## 弯道 6  P1 特殊场景

In [12]:
point_init.position.x = 10.0
point_init.position.y = 6.0
point_init.theta = 0.2
point_init.velocity = 5.67
point_init.acceleration  = 0.5

path = str(current_directory)+str("/../env_simulator/data/curva_lanes_case_2.json")
motion.init(path,point_init)

motion.execute_tree()

show_motion_tree_case(motion)

---- loading straight_case_1 case ----
trajectory tree branch size is:  1
---- loading relative 0 lane 
Field 'road_edges' does not exist.
 r_4 is : 1.13033
j_opt: 63.8194
------- best alpha: 1
------- best alpha: 1
----------enable lat 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 0.683013
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
------- best alpha: 0.683013
------- best alpha: 0.092296
 iter times: 25
 stop search 
 **************** ilqr tree time: 44.6564 ms 
 tree trajectory finish


Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...